In [1]:
# Instalar otras dependencias para ayudarnos
#!pip install tensorflow
#!pip install -q "tqdm>=4.36.1"
# !pip install tensorflow-addons==0.22.0


In [2]:
# importar librerías
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
import tqdm
# import tensorflow_addons as tfa
print(tf.__version__)

2024-04-23 16:46:33.569143: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 16:46:33.591886: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 16:46:33.915170: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.16.1


In [3]:
# Cargamos la data
data = pd.read_csv("data.csv", header=0)
# Hacemos un shuffle de los rows
data = data.sample(frac = 1)
data.head(10)

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
25,54.124920,26.650489,35.329747,27.474432,121.447011,1.571205,Abnormal
134,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134,Abnormal
258,40.349296,10.194748,37.967747,30.154548,128.009927,0.458901,Normal
91,70.952728,20.159931,62.859109,50.792797,116.177932,32.522331,Abnormal
152,63.404481,14.115327,48.136806,49.289153,111.916008,31.784495,Abnormal
11,31.232387,17.715819,15.500000,13.516568,120.055399,0.499751,Abnormal
36,32.090987,6.989378,35.998198,25.101609,132.264735,6.413428,Abnormal
247,49.828135,16.736435,28.000000,33.091700,121.435558,1.913307,Normal
51,74.433593,41.557331,27.700000,32.876262,107.949304,5.000089,Abnormal
141,89.504947,48.903653,72.003423,40.601295,134.634291,118.353370,Abnormal


In [4]:
#Creamos una lista de las posibles clases y su representación inversa
class_names = {
    "Abnormal": 0,
    "Normal": 1
}
reversed_class_names = {v: k for k, v in class_names.items()}
data['class'] = data['class'].map(class_names)
data.head(10)



,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
25,54.124920,26.650489,35.329747,27.474432,121.447011,1.571205,0
134,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134,0
258,40.349296,10.194748,37.967747,30.154548,128.009927,0.458901,1
91,70.952728,20.159931,62.859109,50.792797,116.177932,32.522331,0
152,63.404481,14.115327,48.136806,49.289153,111.916008,31.784495,0
11,31.232387,17.715819,15.500000,13.516568,120.055399,0.499751,0
36,32.090987,6.989378,35.998198,25.101609,132.264735,6.413428,0
247,49.828135,16.736435,28.000000,33.091700,121.435558,1.913307,1
51,74.433593,41.557331,27.700000,32.876262,107.949304,5.000089,0
141,89.504947,48.903653,72.003423,40.601295,134.634291,118.353370,0


In [5]:
#Definimos los porcentajes de entrenamiento:

dataset_percent_for_training = 0.8
dataset_percent_for_testing = 0.2
data_count = len(data)
training_count = int((data_count * dataset_percent_for_training))
testing_count = int(data_count * dataset_percent_for_testing)

# Creamos los sets de entrenamiento y de pruebas
training_data = data.iloc[:, :5][:training_count]
training_labels = data.iloc[:, 6][:training_count]

testing_data = data.iloc[:, :5][training_count:training_count + testing_count]
testing_labels = data.iloc[:, 6][training_count:training_count + testing_count]
print(testing_labels.head(10))

207    0
23     0
106    0
174    0
140    0
254    1
122    0
115    0
30     0
119    0
Name: class, dtype: int64


In [ ]:
# Creamos una red neuronal. Esta red neuronal va a tener 6 neuronas iniciales,
# para poder ser excitadas por las señales de las columnas en la data.
# Tendrá una sola capa ya que las 6 neuronas tienen un valor "relevante" para el final.
# Habrá una sola neurona final que clasifique el resultado como Normal (1) o Anormal (0)
test_acc = 0
while test_acc < 0.9:
    model = keras.Sequential([
        keras.layers.Dense(6, activation='softmax'),
        keras.layers.Dense(2)
    ])

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.0007
    )
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=['accuracy'])
    #Definimos la cantidad de epocs de entrenamiento.
    epochs = 150


    #Comenzamos el entrenamiento
    model.fit(training_data, training_labels, epochs=epochs, verbose=0)                         
    test_loss, test_acc = model.evaluate(testing_data, testing_labels, verbose=2)
for x in range(10):
    !say "Alert!"
    message = f"Created a model with {test_acc * 100} percent accuracy"
    !say {message}

2024-04-23 16:46:34.506153: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 16:46:34.526037: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 16:46:34.527563: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

2/2 - 0s - 133ms/step - accuracy: 0.6774 - loss: 0.6931
2/2 - 0s - 84ms/step - accuracy: 0.6774 - loss: 0.6931
2/2 - 0s - 85ms/step - accuracy: 0.3387 - loss: 0.6904
2/2 - 0s - 85ms/step - accuracy: 0.3710 - loss: 0.6931
2/2 - 0s - 85ms/step - accuracy: 0.6613 - loss: 1.6355
2/2 - 0s - 86ms/step - accuracy: 0.6774 - loss: 0.6288
2/2 - 0s - 85ms/step - accuracy: 0.6290 - loss: 0.6931
2/2 - 0s - 84ms/step - accuracy: 0.6774 - loss: 0.6931
2/2 - 0s - 84ms/step - accuracy: 0.6774 - loss: 0.6931
2/2 - 0s - 86ms/step - accuracy: 0.6613 - loss: 0.9350
2/2 - 0s - 86ms/step - accuracy: 0.6774 - loss: 0.6243
2/2 - 0s - 91ms/step - accuracy: 0.6613 - loss: 0.9299
2/2 - 0s - 86ms/step - accuracy: 0.6774 - loss: 0.6964
2/2 - 0s - 85ms/step - accuracy: 0.3387 - loss: 0.6931
2/2 - 0s - 85ms/step - accuracy: 0.6774 - loss: 0.6931
2/2 - 0s - 91ms/step - accuracy: 0.6774 - loss: 0.6931
2/2 - 0s - 88ms/step - accuracy: 0.6613 - loss: 0.8958
2/2 - 0s - 85ms/step - accuracy: 0.6452 - loss: 0.8844
2/2 - 0s 

In [ ]:
#Evaluemos el modelo
print('\nTest accuracy: ', test_acc)

In [ ]:
#Guardemos el modelo

model_path = 'models/trained_model-09.keras'
model.save(model_path)

In [ ]:
#Hagamos predicciones
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(testing_data)

slice = 10
end = slice + 30
selected_predictions = predictions[slice:end]
selected_actual = testing_labels[slice:end]

fig, axs = plt.subplots(5, 4, figsize=(15, 20))

for i, ax in enumerate(axs.flat):
    # Plotear el valor predicho
    ax.bar(reversed_class_names.values(), selected_predictions[i], alpha= 0.5, label='Predicho')
    # Plotear el valor real
    actual_values = [0] * len(reversed_class_names)
    actual_values[selected_actual.iloc[i]] = 1
    ax.bar(reversed_class_names.values(), actual_values, alpha=0.5, label='Real')
    ax.set_title(f'Predicción {i + 1}')
    ax.set_ylabel('Confianza')
    ax.set_ylim([0, 1])
    ax.legend()
plt.tight_layout()
plt.show()